In [1]:
import random
import pandas as pd
import numpy as np
import sklearn
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import requests
from io import BytesIO
from scipy import stats
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import tensorflow as tf
import math
import cv2


Using TensorFlow backend.


In [2]:
# dataframes - original data
case_path = './data/thyroidCSV/cases.csv'
coord_path = './data/thyroidCSV/coors.csv'
img_path = './data/thyroidCSV/images.csv'

case_df = pd.read_csv(case_path)
coord_df = pd.read_csv(coord_path)
img_df = pd.read_csv(img_path)


In [3]:
# path to all images in a single folder
all_img_path = './data/all_images_by_coord_id/'
img_ids = ['img_'+str(num) for num in range(1, len(coord_df)+1)]


In [4]:
# open images. convert to array and radd border to make square imgs
imgs = [Image.open(all_img_path+im_id) for im_id in img_ids]
imgs = [np.array(im) for im in imgs]
imgs = [cv2.copyMakeBorder(im,0,200,0,0,cv2.BORDER_CONSTANT) for im in imgs]


In [5]:
# convert back to img files
imgs = [Image.fromarray(im) for im in imgs]

In [6]:

new_size = (416, 416)
for im in imgs:
    im.thumbnail(new_size)  

In [7]:
new_imgs = [im.resize(new_size) for im in imgs]
imgs = new_imgs

In [19]:
# check for uniformity
sizes = [im.size for im in imgs]
set(sizes)


779


In [18]:
# put resized imgs in folder
i = 0
new_path = []
basepath = 'data/resized_imgs/'
for im in imgs:
    im.save(basepath+img_ids[i], "PNG")
    new_path.append(basepath+img_ids[i])
    i+=1


779


In [22]:
coord_df['new_path'] = new_path
coord_df.sample(3)

,id,image_id,x,y,w,h,new_path
354,355,299,165,64,243,170,data/resized_imgs/img_355
749,750,596,413,146,46,47,data/resized_imgs/img_750
68,69,68,260,89,77,82,data/resized_imgs/img_69


In [10]:
# create new filename col data
files = img_df['filename'].values
filenames = ['/data/'+ x for x in files]



In [11]:
# new column in img df
img_df['filename'] = filenames


In [12]:
img_df.head(3)

,id,case_id,filename,mark
0,1,1,/data/cimalab/bening/545/1.jpg,"[{""points"": [{""x"": 413, ""y"": 91}, {""x"": 406, ""..."
1,2,1,/data/cimalab/bening/545/2.jpg,"[{""points"": [{""x"": 291, ""y"": 76}, {""x"": 281, ""..."
2,3,5,/data/cimalab/bening/549/1.jpg,"[{""points"": [{""x"": 292, ""y"": 123}, {""x"": 293, ..."


In [13]:
case_df.head(3)


,id,num,cancer,age,sex,composition,echogenicity,margins,calcification,tirads,reportbacaf,reporteco
0,1,545,NaN,72.0,F,solid,marked hypoechogenecity,ill- defined,NaN,5,NaN,NaN
1,2,546,NaN,63.0,F,solid,isoechogenicity,microlobulated,NaN,4c,NaN,NaN
2,3,547,NaN,61.0,F,spongiform appareance,hypoechogenecity,well defined smooth,NaN,4a,NaN,NaN


In [38]:
###### join filepaths to coord_df by img_df id column
#coord_df['img_file'] = [img_df[img_df['id'] == n]['filename'].values[0] for n in coord_df['image_id']]




779

In [37]:
im_subset = img_df[['id', 'case_id']]
# coord_df.head(3)


618

In [16]:
img_df.head(3)

,id,case_id,filename,mark
0,1,1,/data/cimalab/bening/545/1.jpg,"[{""points"": [{""x"": 413, ""y"": 91}, {""x"": 406, ""..."
1,2,1,/data/cimalab/bening/545/2.jpg,"[{""points"": [{""x"": 291, ""y"": 76}, {""x"": 281, ""..."
2,3,5,/data/cimalab/bening/549/1.jpg,"[{""points"": [{""x"": 292, ""y"": 123}, {""x"": 293, ..."


In [17]:
# Create splits - use ONCE


# new_df = coord_df.copy()
# train, validate, test = np.split(new_df.sample(frac=1), [int(.8*len(new_df)), int(.9*len(new_df))])

In [18]:
# write new csvs with train valid test splits

# train.to_csv('./data/train_df.csv')
# validate.to_csv('./data/validate_df.csv')
# test.to_csv('./data/test_df.csv')

In [19]:
# read new CSVs

train = pd.read_csv('./data/train_df.csv')
validate = pd.read_csv('./data/validate_df.csv')
test = pd.read_csv('./data/test_df.csv')



In [35]:
num_list = train.image_id.values
train.head(5)


train.image_id.duplicated
#train[train['image_id']==100]

AttributeError: 'function' object has no attribute 'sum'

In [21]:
# VOC format
import xml.etree.ElementTree as ET
import xml.dom.minidom

def prep_for_VOC(x, y, w, h, folder, filename):
    if 'train' in folder:
        folder = 'train'
    if 'valid' in folder:
        folder = 'validate'
    if 'test' in folder:
        folder = 'test'
        
    # scale data to fit new sizes
    x = round(x*.75 - 4)
    y = round(y*.75 - 4)
    w = round(w*.75 - 4)
    h = round(h*.75 - 4)
    
    root = ET.Element('annotation')
    fol_elem = ET.SubElement(root, 'folder')
    fol_elem.text = folder
    file_elem = ET.SubElement(root, 'filename')
    file_elem.text = filename
    path_elem = ET.SubElement(root,'path')
    path_elem.text = ('/Users/kathleensullivan/PythonNYCDSA/Machine_Learning/capstone/data/'+folder+'/'+filename)
    src_elem = ET.SubElement(root, 'source')
    db = ET.SubElement(src_elem, 'database')
    db.text=('Unknown')
    sz_elem = ET.SubElement(root, 'size')
    w_elem = ET.SubElement(sz_elem, 'width')
    w_elem.text = str(w)
    h_elem = ET.SubElement(sz_elem, 'height')
    h_elem.text = str(h)
    d_elem = ET.SubElement(sz_elem, 'depth')
    d_elem.text = '3'
    seg_elem = ET.SubElement(root,'segmented')
    seg_elem.text='0'
    obj_elem = ET.SubElement(root, 'object')
    name_elem = ET.SubElement(obj_elem,'name')
    name_elem.text = 'lesion'
    pose_elem = ET.SubElement(obj_elem,'pose')
    pose_elem.text = 'Unspecified'
    trunc_elem = ET.SubElement(obj_elem,'truncated')
    trunc_elem.text = '0'
    dif_elem = ET.SubElement(obj_elem,'difficult')
    dif_elem.text = '0'
    
    bb_elem = ET.SubElement(obj_elem,'bndbox')
    xmin_elem = ET.SubElement(bb_elem,'xmin')
    xmin_elem.text = str(x)
    xmax_elem = ET.SubElement(bb_elem,'xmax')
    xmax_elem.text = str(x + w)
    ymin_elem = ET.SubElement(bb_elem,'ymin')
    ymin_elem.text = str(y)
    ymax_elem = ET.SubElement(bb_elem,'ymax')
    ymax_elem.text = str(y + h)
    return root
    

In [22]:
# DARKNET/AlexeyAB implementation
# def find_center_scaled(x, y, w, h):
#     # pass in x, y, w, h from df.values
#     end_x = x + w
#     end_y = y - h
#     center_x = (x + end_x)/2
#     center_y = (y + end_y)/2
#     # method suggested here: https://github.com/AlexeyAB/Yolo_mark/issues/60
#     scaled_x = center_x/560
#     scaled_y = center_y/360
#     scaled_w = w/560
#     scaled_h = h/360
    
#     return scaled_x, scaled_y, scaled_w, scaled_h


# def write_files(df_name):
#     str_name =[x for x in globals() if globals()[x] is df_name][0]
#     txt_files = []
#     num_list = df_name.image_id.values
#     for num in num_list:
#         df = df_name[df_name.image_id == num]
#         dim = df.values[0]
#         s_x, s_y, s_w, s_h = find_center_scaled(dim[2], dim[3], dim[4], dim[5])
#         f = open('./data/{}_annot/img_{}.txt'.format(str_name, num),"w+")
#         f.write('1 {} {} {} {}'.format(s_x, s_y, s_w, s_h))
#         f.close()
#         txt_files.append('./data/{}/img_{}.txt'.format(str_name, num))
#         img = Image.open(df.img_file.values[0])
#         img.save('./data/'+str_name+'/img_'+str(num)+'.jpg', 'JPEG')
#     f = open('./data/{}.txt'.format(str_name), 'w+')
#     for t in txt_files:
#         f.write(t+'\n')
#     f.close()


In [23]:
from xml.etree.ElementTree import Element, SubElement, Comment, tostring
from xml.dom import minidom


def write_files(df_name):
    str_name =[x for x in globals() if globals()[x] is df_name][0]
    txt_files = []
    num_list = df_name.image_id.values
    for num in num_list:
        df = df_name[df_name.image_id == num]
        dim = df.values[0]
        
        tree = ET.tostring(prep_for_VOC(dim[2],dim[3], dim[4], dim[5], str_name+'_annot', 'img_{}.png'.format(num)) )
        xmlstr = minidom.parseString(tree)
        xmlstr = xmlstr.toprettyxml() 
        xmlstr = xmlstr[22:].strip()
        with open('./data/{}_annot/img_{}.xml'.format(str_name, num), 'w') as f:
             f.write(xmlstr)
        txt_files.append('./data/{}/img_{}.xml'.format(str_name, num))
        img = Image.open(df.new_path.values[0]).convert('LA')
        
        img.save('./data/'+str_name+'/img_'+str(num)+'.png', 'PNG')
    f = open('./data/{}.txt'.format(str_name), 'w+')
    for t in txt_files:
        f.write(t+'\n')
    f.close()
    

In [24]:
# random.seed(5)
# train=train.copy(deep=True)
# train = train.sample(30)
# validate = validate.sample(10)
# test = test.sample(10)


In [25]:
### write partial files for testing
# write_files(train)
# write_files(validate)
# write_files(test)

In [26]:
#### full files
write_files(validate)
write_files(train)
write_files(test)


In [27]:
# Create ground truth files
def scale(num):
    return round(num*.75-4)


def write_gr_truth(df_name):
    str_name =[x for x in globals() if globals()[x] is df_name][0]
    txt_files = []
    num_list = df_name.image_id.values
    for num in num_list:
        df = df_name[df_name.image_id == num]
        dim = df.values[0]
        left, top, right, bottom = scale(dim[2]), scale(dim[3]), scale(dim[4] + dim[2]), scale(dim[5]+dim[3])
        f = open('./data/{}_annot/img_{}.txt'.format(str_name, num),"w+")
        f.write('lesion', {} {} {} {}'.format(left, top, right, bottom)
        f.close()
        txt_files.append('./data/{}/img_{}.txt'.format(str_name, num))
        img = Image.open(df.img_file.values[0])
        img.save('./Object-Detection-Metrics/groundtruth/'+str_name+'/img_'+str(num)+'.png', 'PNG')
    f = open('./data/{}.txt'.format(str_name), 'w+')
    for t in txt_files:
        f.write(t+'\n')
    f.close()

SyntaxError: invalid syntax (<ipython-input-27-b447e57ae6de>, line 15)